# Data Preparation

## Import Libraries

In [1]:
import pandas as pd 
import numpy as np 
import scipy as sp 

import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession

# Importar funciones necesarias
from pyspark.sql.functions import col, to_date, weekofyear,year, month, dayofmonth, sum
from pyspark.sql.functions import col, sum as spark_sum
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import IntegerType
# Puedes obtener estadísticas específicas para una columna
from pyspark.sql.functions import mean, min, max
from pyspark.sql.functions import approx_count_distinct
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Import Data

This dataset contains the full LendingClub data available from [their site](https://www.lendingclub.com). There are separate files for accepted and rejected loans. The accepted loans also include the FICO scores, which can only be downloaded when you are signed in to LendingClub and download the data (https://www.kaggle.com/datasets/wordsforthewise/lending-club).



In [2]:

# Crear la sesión de Spark
spark = SparkSession.builder.appName("AppLending").getOrCreate()

# Rutas de los archivos CSV comprimidos
accepted_r = 'hdfs://namenode:8020/datasets/raw/accepted_2007_to_2018Q4.csv.gz'
#accepted_df = spark.read.csv("hdfs://namenode:8020/datasets/raw/accepted_2007_to_2018Q4.csv.gz", header=True, inferSchema=True)

# Leer los archivos CSV comprimidos usando PySpark
accepted_df = spark.read.option("header", "true").option("inferSchema", "true").csv(accepted_r)


In [4]:
accepted_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: double (nullable = true)
 |-- funded_amnt: double (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 